In [17]:


import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



Spark NLP version 3.1.0
Apache Spark version: 3.0.2


In [18]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [19]:
import pyspark.sql.functions as F

training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("pos"),
        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


+--------+---+---------+
|   token|pos|ner_label|
+--------+---+---------+
|    make|  0|      Act|
|     can|  0|      Act|
|      do|  0|      Act|
|    read|  0|      Act|
|   write|  0|      Act|
|    talk|  0|      Act|
|    want|  0|      Act|
|    cook|  0|      Act|
|    brew|  0|      Act|
|   clean|  0|      Act|
|   brush|  0|      Act|
|    care|  0|      Act|
|    walk|  0|      Act|
|      go|  0|      Act|
|navigate|  0|      Act|
|activity|  0|      Act|
|   write|  0|      Act|
|    move|  0|      Act|
|    fall|  0|      Act|
|      up|  0|      Act|
+--------+---+---------+
only showing top 20 rows



In [20]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [21]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')
embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(100)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      document_assembler, 
      tokenizer,
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [22]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.write().overwrite().save('outputs/ner_wiki_glove100d_en')

2022-10-13 20:15:13.927300: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

Training started - total epochs: 100 - lr: 0.003 - batch size: 32 - labels: 7 - chars: 24 - training examples: 130
Epoch 1/100 started, lr: 0.003, dataset size: 130
Epoch 1/100 - 0.60s - loss: 25.787727 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.16s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 10	 2	 3	 0.8333333	 0.7692308	 0.8
Act	 14	 14	 7	 0.5	 0.6666667	 0.57142854
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 0	 20	 1.0	 0.5121951	 0.67741936
Obj	 13	 17	 0	 0.43333334	 1.0	 0.60465115
Pos	 0	 0	 2	 0.0	 0.0	 0.0
tp: 58 fp: 33 fn: 33 labels: 6
Macro-average	 prec: 0.4611111, rec: 0.49134874, f1: 0.47574997
Micro-average	 prec: 0.63736266, rec: 0.63736266, f1: 0.63736266
Epoch 2/100 started, lr: 0.0029850747, dataset size: 130
Epoch 2/100 - 0.07s - loss: 45.169243 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 3	 0	 0.8125	 1.0	 0.8

Epoch 14/100 - 0.07s - loss: 1.4065363 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 17	 4	 4	 0.8095238	 0.8095238	 0.8095238
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 38	 1	 3	 0.974359	 0.9268293	 0.95
Obj	 9	 7	 4	 0.5625	 0.6923077	 0.62068963
Pos	 0	 1	 2	 0.0	 0.0	 0.0
tp: 77 fp: 14 fn: 14 labels: 6
Macro-average	 prec: 0.5577305, rec: 0.5714435, f1: 0.5645037
Micro-average	 prec: 0.84615386, rec: 0.84615386, f1: 0.84615386
Epoch 15/100 started, lr: 0.0028037382, dataset size: 130
Epoch 15/100 - 0.07s - loss: 0.9012612 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 17	 4	 4	 0.8095238	 0.8095238	 0.8095238
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 38	 1	 3	 0.974359	 0.9268293	 0.95
Obj	 9	 7	 4	 0.5625	 0.6923077	 0.62068963
Pos	 0	 1	 2	 0.0	

Epoch 29/100 - 0.07s - loss: 0.09680966 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 17	 3	 4	 0.85	 0.8095238	 0.82926834
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 39	 2	 2	 0.9512195	 0.9512195	 0.9512195
Obj	 10	 5	 3	 0.6666667	 0.7692308	 0.71428573
Pos	 0	 1	 2	 0.0	 0.0	 0.0
tp: 79 fp: 12 fn: 12 labels: 6
Macro-average	 prec: 0.57798105, rec: 0.588329, f1: 0.58310914
Micro-average	 prec: 0.8681319, rec: 0.8681319, f1: 0.8681319
Epoch 30/100 started, lr: 0.0026200875, dataset size: 130
Epoch 30/100 - 0.07s - loss: 0.097573884 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 17	 3	 4	 0.85	 0.8095238	 0.82926834
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 39	 2	 2	 0.9512195	 0.9512195	 0.9512195
Obj	 10	 5	 3	 0.6666667	 0.7692308	 0.71428573
Pos	 

Epoch 44/100 - 0.07s - loss: 0.023838434 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 17	 3	 4	 0.85	 0.8095238	 0.82926834
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 39	 1	 2	 0.975	 0.9512195	 0.962963
Obj	 11	 5	 2	 0.6875	 0.84615386	 0.7586207
Pos	 0	 1	 2	 0.0	 0.0	 0.0
tp: 80 fp: 11 fn: 11 labels: 6
Macro-average	 prec: 0.5854167, rec: 0.60114956, f1: 0.5931788
Micro-average	 prec: 0.8791209, rec: 0.8791209, f1: 0.8791209
Epoch 45/100 started, lr: 0.0024590164, dataset size: 130
Epoch 45/100 - 0.07s - loss: 0.032545663 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 17	 3	 4	 0.85	 0.8095238	 0.82926834
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 39	 1	 2	 0.975	 0.9512195	 0.962963
Obj	 11	 5	 2	 0.6875	 0.84615386	 0.7586207
Pos	 0	 1	 2	 0.0	 0

Epoch 59/100 - 0.08s - loss: 0.05880758 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 17	 5	 4	 0.77272725	 0.8095238	 0.79069763
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 39	 1	 2	 0.975	 0.9512195	 0.962963
Obj	 9	 5	 4	 0.64285713	 0.6923077	 0.6666667
Pos	 0	 1	 2	 0.0	 0.0	 0.0
tp: 78 fp: 13 fn: 13 labels: 6
Macro-average	 prec: 0.5650974, rec: 0.57550853, f1: 0.57025546
Micro-average	 prec: 0.85714287, rec: 0.85714287, f1: 0.85714287
Epoch 60/100 started, lr: 0.0023166025, dataset size: 130
Epoch 60/100 - 0.07s - loss: 0.0315895 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 17	 5	 4	 0.77272725	 0.8095238	 0.79069763
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 39	 1	 2	 0.975	 0.9512195	 0.962963
Obj	 9	 5	 4	 0.64285713	 0.6923077	 0.6666667
Pos

Epoch 74/100 - 0.07s - loss: 0.03008689 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 1	 0	 0.9285714	 1.0	 0.9629629
Act	 17	 3	 4	 0.85	 0.8095238	 0.82926834
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 38	 1	 3	 0.974359	 0.9268293	 0.95
Obj	 11	 6	 2	 0.64705884	 0.84615386	 0.73333335
Pos	 0	 1	 2	 0.0	 0.0	 0.0
tp: 79 fp: 12 fn: 12 labels: 6
Macro-average	 prec: 0.5666649, rec: 0.5970845, f1: 0.5814771
Micro-average	 prec: 0.8681319, rec: 0.8681319, f1: 0.8681319
Epoch 75/100 started, lr: 0.002189781, dataset size: 130
Epoch 75/100 - 0.07s - loss: 0.007793634 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 17	 4	 4	 0.8095238	 0.8095238	 0.8095238
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 39	 1	 2	 0.975	 0.9512195	 0.962963
Obj	 11	 5	 2	 0.6875	 0.84615386	 0.7586207
Pos

Epoch 89/100 - 0.07s - loss: 0.029641515 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 16	 4	 5	 0.8	 0.7619048	 0.78048784
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 40	 1	 1	 0.9756098	 0.9756098	 0.9756098
Obj	 10	 5	 3	 0.6666667	 0.7692308	 0.71428573
Pos	 0	 1	 2	 0.0	 0.0	 0.0
tp: 79 fp: 12 fn: 12 labels: 6
Macro-average	 prec: 0.57371277, rec: 0.5844576, f1: 0.57903534
Micro-average	 prec: 0.8681319, rec: 0.8681319, f1: 0.8681319
Epoch 90/100 started, lr: 0.0020761248, dataset size: 130
Epoch 90/100 - 0.07s - loss: 0.007585004 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 16	 4	 5	 0.8	 0.7619048	 0.78048784
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 40	 1	 1	 0.9756098	 0.9756098	 0.9756098
Obj	 10	 5	 3	 0.6666667	 0.7692308	 0.71428573
Pos	 

2022-10-13 20:15:23.573431: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

CPU times: user 105 ms, sys: 48.3 ms, total: 153 ms
Wall time: 13 s


In [23]:
import pyspark.sql.functions as F
predictions = ner_model.transform(testing_data) 
predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
           .select(F.expr("cols['0']").alias("token"),
                   F.expr("cols['1']").alias("ground_truth"),
                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

+--------+------------+----------+
|token   |ground_truth|prediction|
+--------+------------+----------+
|I       |0           |0         |
|want    |Act         |Act       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
|location|Loc         |Loc       |
|I       |0           |0         |
|want    |Act         |Act       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
|kitchen |Loc         |Loc       |
|I       |0           |0         |
|want    |Act         |Act       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
+--------+------------+----------+
only showing top 20 rows



In [24]:
from sklearn.metrics import classification_report
 
preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()
 
print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       138
         Act       1.00      1.00      1.00        42
         Loc       1.00      1.00      1.00        36

    accuracy                           1.00       216
   macro avg       1.00      1.00      1.00       216
weighted avg       1.00      1.00      1.00       216



In [25]:
#import shutil

#shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

In [26]:
text_list = [
    """Navagate to kitchen"""
]


In [27]:
from sparknlp.base import LightPipeline

light = LightPipeline(ner_model)
light.annotate("navigate to kitchen")


{'document': ['navigate to kitchen'],
 'ner_chunk': ['navigate to kitchen'],
 'token': ['navigate', 'to', 'kitchen'],
 'ner': ['Act', '0', 'Loc'],
 'embeddings': ['navigate', 'to', 'kitchen']}

In [28]:
light.annotate("Navigate to kitchen")


{'document': ['Navigate to kitchen'],
 'ner_chunk': ['kitchen'],
 'token': ['Navigate', 'to', 'kitchen'],
 'ner': ['0', '0', 'Loc'],
 'embeddings': ['Navigate', 'to', 'kitchen']}

In [34]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    print(light.annotate(text))
    text = input("Enter New Text\n")

Enter Testing Text
test
{'document': ['test'], 'ner_chunk': ['test'], 'token': ['test'], 'ner': ['Obj'], 'embeddings': ['test']}
Enter New Text
what am i
{'document': ['what am i'], 'ner_chunk': ['what am i'], 'token': ['what', 'am', 'i'], 'ner': ['Neg', 'Neg', 'Neg'], 'embeddings': ['what', 'am', 'i']}
Enter New Text
Yes
{'document': ['Yes'], 'ner_chunk': ['Yes'], 'token': ['Yes'], 'ner': ['Pos'], 'embeddings': ['Yes']}
Enter New Text
No
{'document': ['No'], 'ner_chunk': ['No'], 'token': ['No'], 'ner': ['Neg'], 'embeddings': ['No']}
Enter New Text
Navigate
{'document': ['Navigate'], 'ner_chunk': [], 'token': ['Navigate'], 'ner': ['0'], 'embeddings': ['Navigate']}
Enter New Text
navigate
{'document': ['navigate'], 'ner_chunk': ['navigate'], 'token': ['navigate'], 'ner': ['Act'], 'embeddings': ['navigate']}
Enter New Text
where am I
{'document': ['where am I'], 'ner_chunk': ['where am I'], 'token': ['where', 'am', 'I'], 'ner': ['Obj', 'Obj', '0'], 'embeddings': ['where', 'am', 'I']}
Ent

KeyboardInterrupt: Interrupted by user

In [29]:
#empty_df = spark.createDataFrame([['']]).toDF('text')
#pipeline_model = ner_pipeline.fit(empty_df)
#df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
#result = pipeline_model.transform(df)

AnalysisException: cannot resolve '`label`' given input columns: [document, embeddings, text, token];;
'Project ['label, 'sentence, token#763, embeddings#774]
+- Sample 0.2, 1.0, false, 6301249005639537257
   +- Sort [text#754 ASC NULLS FIRST], false
      +- Project [text#754, document#758, token#763, embeddings#768 AS embeddings#774]
         +- Project [text#754, document#758, token#763, UDF(array(document#758, token#763)) AS embeddings#768]
            +- Project [text#754, document#758, UDF(array(document#758)) AS token#763]
               +- Project [text#754, UDF(text#754) AS document#758]
                  +- Project [_1#752 AS text#754]
                     +- LogicalRDD [_1#752], false


In [30]:




#from sparknlp_display import NerVisualizer



#result = predictions

#NerVisualizer().display(
#    result = result.collect()[0],
#    label_col = 'ner_chunk',
#    document_col = 'document'
#)




